# [Day 16: Packet Decoder](https://adventofcode.com/2021/day/16)

In [1]:
import dataclasses as dc
import math
from typing import Generator

## Part 1

In [2]:
example_data_part1 = {
    ("620080001611562C8802118E34", 12),
    ("8A004A801A8002F478", 16),
    ("C0015000016115A2E0802F182340", 23),
    ("A0016C880162017C3686B18A3D4780", 31),
}

In [3]:
def bit_stream_generator(input_data: str) -> Generator[str, None, None]:
    for hex_start in range(0, len(input_data), 2):
        for byte_as_bits in [('00000000' + f"{int(input_data[hex_start:hex_start+2], 16):b}")[-8:]]:
            for bit in byte_as_bits:
                yield bit


def get_bits(bit_stream: Generator[str, None, None], n: int) -> int:
    return int("".join([next(bit_stream) for i in range(n)]), 2)


def read_literal(bit_stream: Generator[str, None, None]) -> [int, int, int]:
    literal = 0
    bit_counter = 0
    not_done = True
    while not_done:
        # read 5 bits
        block = get_bits(bit_stream, 5)
        bit_counter += 5
        # when first bit is 0: last block
        if block < 0x10:
            not_done = False
        # calculate new literal
        literal = literal * 0x10 + (block & 0xF)
    return 0, literal, bit_counter


def read_operator(bit_stream: Generator[str, None, None]) -> [int, int, int]:
    ## first bit is length type id: 0=bit counter, 1=packet counter
    length_type_id = get_bits(bit_stream, 1)
    bit_counter = 1
    
    # read the counter (either bits or packets)
    if length_type_id == 0:
        # type 0: next 15 bits is length in bits of sub-packets
        counter = get_bits(bit_stream, 15)
        bit_counter += 15
    else:
        # type 1: next 11 bits is the number of sub-packets
        counter = get_bits(bit_stream, 11)
        bit_counter += 11

    # process all sub packets
    version_sum = 0
    values = []
    while counter > 0: 
        sub_version_sum, value, bits_read = read_packet(bit_stream)
        values.append(value)
        version_sum += sub_version_sum
        bit_counter += bits_read
        if length_type_id == 0:
            # type 0: count bits
            counter -= bits_read
        else:
            # type 1: count packets
            counter -= 1
    
    return version_sum, values, bit_counter


def operate(type_id: int, values: list[int]) -> int:
    if type_id == 0:  # sum
        return sum(values)
    elif type_id == 1:  # product
        return math.prod(values)
    elif type_id == 2:  # minimum
        return min(values)
    elif type_id == 3:  # maximum
        return max(values)
    elif type_id == 5:  # greater
        return 1 if values[0] > values[1] else 0
    elif type_id == 6:  # less
        return 1 if values[0] < values[1] else 0
    elif type_id == 7:  # equal
        return 1 if values[0] == values[1] else 0
    else:
        return 0  # shouldn't happen, just in case
    
    
def read_packet(bit_stream: Generator[str, None, None]) -> [int, int, int]:
    # first 3 bits: version
    version = get_bits(bit_stream, 3)
    # next 3 bits: packet type id
    type_id = get_bits(bit_stream, 3)
    bit_counter = 6
    
    # process one packet
    if type_id == 4:
        # type 4: literal value packet
        version_sum, value, bits_read = read_literal(bit_stream)
        bit_counter += bits_read
    else:
        # type != 4: operator packet
        version_sum, values, bits_read = read_operator(bit_stream)
        bit_counter += bits_read
        value = operate(type_id, values)

    return version+version_sum, value, bit_counter

In [4]:
for example_data, expected in example_data_part1:
    version_sum, value, bits_read = read_packet(bit_stream_generator(example_data))
    print(f"Check part 1 ({example_data}): {version_sum == expected}")

Check part 1 (8A004A801A8002F478): True
Check part 1 (A0016C880162017C3686B18A3D4780): True
Check part 1 (C0015000016115A2E0802F182340): True
Check part 1 (620080001611562C8802118E34): True


In [5]:
with open(r"..\data\Day 16 input.txt", "r") as fh_in:
    input_data = fh_in.readline().strip()
print(f"Input check: {len(input_data) == 1370}")

Input check: True


In [6]:
print(f"Answer part 1: {read_packet(bit_stream_generator(input_data))[0]}")

Answer part 1: 877


## Part 2

In [7]:
example_data_part2 = {
    ("C200B40A82", 3),
    ("04005AC33890", 54),
    ("880086C3E88112", 7),
    ("CE00C43D881120", 9),
    ("D8005AC2A8F0", 1),
    ("F600BC2D8F", 0),
    ("9C005AC2F8F0", 0),
    ("9C0141080250320F1802104A08", 1),
}

In [8]:
for example_data, expected in example_data_part2:
    version_sum, value, bits_read = read_packet(bit_stream_generator(example_data))
    print(f"Check part 2 ({example_data}): {value == expected}")

Check part 2 (D8005AC2A8F0): True
Check part 2 (880086C3E88112): True
Check part 2 (9C0141080250320F1802104A08): True
Check part 2 (9C005AC2F8F0): True
Check part 2 (C200B40A82): True
Check part 2 (F600BC2D8F): True
Check part 2 (04005AC33890): True
Check part 2 (CE00C43D881120): True


In [9]:
print(f"Answer part 2: {read_packet(bit_stream_generator(input_data))[1]}")

Answer part 2: 194435634456
